In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

### Code from Visualization notebook to create wages data frame

In [2]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
    
url = urlopen("https://www.cengage.com/aise/economics/wooldridge_3e_datasets/statafiles.zip")

with ZipFile(BytesIO(url.read())) as zipped:
    file = zipped.open("WAGE1.DTA")

stata = pd.read_stata(file, iterator=True)
wages = stata.read()

wages['area'] = 0
for i, col in enumerate(['northcen', 'south', 'west']):
    wages['area'] += (i+1) * wages[col]
wages['area'] = wages.area.map({0: 'northeast', 1: 'northcen', 2: 'south', 3: 'west'})

occupations = wages.columns.to_list()[12:18] 
wages['occup'] = 0
for i, col in enumerate(occupations):
    wages['occup'] += (i+1) * wages[col]
dct = {0: 'other'}
dct.update({(i+1): occupations[i] for i in range(6)})
wages['occup'] = wages.occup.map(dct)

wages = wages[
    [
        'wage', 
        'educ', 
        'exper', 
        'tenure', 
        'nonwhite', 
        'female', 
        'married',
        'numdep',
        'smsa',
        'area', 
        'occup'
    ]
]
wages.head()

,wage,educ,exper,tenure,nonwhite,female,married,numdep,smsa,area,occup
0,3.10,11,2,0,0,1,0,2,1,west,other
1,3.24,12,22,2,0,1,1,3,1,west,services
2,3.00,11,2,0,0,0,0,2,0,west,trade
3,6.00,8,44,28,0,0,1,0,1,west,other
4,5.30,12,7,2,0,0,1,1,0,west,other


### Basic regression

In [3]:
result = smf.ols("wage ~ educ", data=wages).fit()
# result.summary()

### Heteroskedasticity consistent standard errors

In [4]:
result = smf.ols("wage ~ educ", data=wages).fit()
result = result.get_robustcov_results(cov_type='HC3')
# result.summary()

### Saving output to Excel

In [5]:
table = result.summary().tables[1]
pd.DataFrame(table).to_excel('test.xlsx', header=False, index=False)

### Saving output to latex

In [6]:
stargazer = Stargazer([result])
tex = stargazer.render_latex()
with open("test.tex", "w") as file:
    file.write(tex)

# print(tex)

ValueError: Please use trained OLS models as inputs

### Multivariate

In [ ]:
result = smf.ols("wage ~ educ+exper+tenure+numdep", data=wages).fit()
result = result.get_robustcov_results(cov_type='HC3')
# result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     58.47
Date:                Sun, 21 Aug 2022   Prob (F-statistic):           9.13e-41
Time:                        19:13:09   Log-Likelihood:                -1335.5
No. Observations:                 526   AIC:                             2681.
Df Residuals:                     521   BIC:                             2702.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.3550      0.788     -4.260      0.000      -4.902      -1.808
educ           0.6198      0.053     11.733      0.000       0.516       0.724
exper          0.0248      0.012      2.044      0.041       0.001       0.049
tenure         0.1682      0.022      7.781      0.000       0.126       0.211
numdep         0.1765      0.110      1.604      0.109      -0.040       0.393
==============================================================================
Omnibus:                      177.662   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              631.337
Skew:                           1.545   Prob(JB):                    8.07e-138
Kurtosis:                       7.389   Cond. No.                         146.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Dummy and Categorical Variables

We could do C(area) and C(occup) but this is unnecessary for categorical text variables.  We might want to treat numdep as numerical, but using C(numdep) causes it to be treated as categorical (generating dummy variables).

In [ ]:
result = smf.ols(
    "wage ~ educ+exper+tenure+C(numdep)+female+nonwhite+married+smsa+area+occup", 
    data=wages
).fit()
result = result.get_robustcov_results(cov_type='HC3')
# result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     17.29
Date:                Sun, 21 Aug 2022   Prob (F-statistic):           2.42e-48
Time:                        19:56:32   Log-Likelihood:                -1284.9
No. Observations:                 526   AIC:                             2616.
Df Residuals:                     503   BIC:                             2714.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.0250      0.962     -2.104      0.036      -3.916      -0.134
C(numdep)[T.1]        0.4160      0.346      1.203      0.229      -0.263       1.095
C(numdep)[T.2]        0.5099      0.359      1.418      0.157      -0.196       1.216
C(numdep)[T.3]        0.7922      0.478      1.656      0.098      -0.148       1.732
C(numdep)[T.4]       -0.1845      0.745     -0.248      0.804      -1.648       1.279
C(numdep)[T.5]        0.8164      1.144      0.714      0.476      -1.431       3.064
C(numdep)[T.6]       -0.6022      2.082     -0.289      0.773      -4.693       3.488
area[T.northeast]     0.5661      0.364      1.554      0.121      -0.150       1.282
area[T.south]         0.2013      0.334      0.602      0.548      -0.456       0.858
area[T.west]          1.0607      0.398      2.662      0.008       0.278       1.843
occup[T.ndurman]     -0.1927      0.703     -0.274      0.784      -1.574       1.188
occup[T.other]        0.6549      0.675      0.970      0.333      -0.672       1.982
occup[T.profserv]    -0.0083      0.656     -0.013      0.990      -1.296       1.280
occup[T.services]    -1.2395      0.717     -1.729      0.084      -2.648       0.169
occup[T.trade]       -1.2105      0.634     -1.909      0.057      -2.456       0.035
occup[T.trcommpu]    -0.5882      0.851     -0.692      0.490      -2.259       1.083
educ                  0.5344      0.054      9.928      0.000       0.429       0.640
exper                 0.0264      0.012      2.177      0.030       0.003       0.050
tenure                0.1211      0.021      5.782      0.000       0.080       0.162
female               -1.8288      0.274     -6.678      0.000      -2.367      -1.291
nonwhite             -0.1434      0.424     -0.338      0.735      -0.976       0.689
married               0.3431      0.295      1.164      0.245      -0.236       0.922
smsa                  0.8540      0.297      2.871      0.004       0.270       1.438
==============================================================================
Omnibus:                      197.727   Durbin-Watson:                   1.848
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              830.153
Skew:                           1.666   Prob(JB):                    5.43e-181
Kurtosis:                       8.174   Cond. No.                         418.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
stargazer = Stargazer([result])
stargazer.covariate_order(['educ', 'exper', 'tenure', 'female', 'nonwhite', 'married', 'smsa'])
tex = stargazer.render_latex()
with open("test.tex", "w") as file:
    file.write(tex)

# print(tex)

ValueError: Please use trained OLS models as inputs

### Multiple models

In [ ]:
mod1 = smf.ols(
    "wage ~ educ+C(numdep)+smsa+area+occup", 
    data=wages
)

mod2 = smf.ols(
    "wage ~ educ+exper+tenure+C(numdep)+smsa+area+occup", 
    data=wages
)

mod3 = smf.ols(
    "wage ~ educ+exper+tenure+female+nonwhite+married+C(numdep)+smsa+area+occup", 
    data=wages
)

results = []
for mod in [mod1, mod2, mod3]: # , mod2, mod3]:
    result = mod.fit()
    result = result.get_robustcov_results(cov_type='HC3')
    results.append(result)

x = [results[0]]
stargazer = Stargazer(x)
stargazer.covariate_order(['educ', 'exper', 'tenure', 'female', 'nonwhite', 'married'])
tex = stargazer.render_latex()
print(tex)

ValueError: Please use trained OLS models as inputs

In [ ]:
results[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     7.019
Date:                Sun, 21 Aug 2022   Prob (F-statistic):           2.09e-15
Time:                        20:25:23   Log-Likelihood:                -1352.6
No. Observations:                 526   AIC:                             2741.
Df Residuals:                     508   BIC:                             2818.
Df Model:                          17                                         
Covariance Type:                  HC3                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.2263      1.116     -1.099      0.272      -3.418       0.966
C(numdep)[T.1]       -0.1365      0.405     -0.337      0.736      -0.932       0.659
C(numdep)[T.2]        0.0453      0.409      0.111      0.912      -0.758       0.849
C(numdep)[T.3]        0.9199      0.594      1.550      0.122      -0.246       2.086
C(numdep)[T.4]       -0.6663      0.520     -1.282      0.201      -1.688       0.355
C(numdep)[T.5]        0.1920      0.971      0.198      0.843      -1.717       2.101
C(numdep)[T.6]        0.2730      0.669      0.408      0.683      -1.042       1.588
area[T.northeast]     0.6420      0.432      1.486      0.138      -0.207       1.491
area[T.south]         0.2035      0.339      0.600      0.549      -0.463       0.870
area[T.west]          0.8794      0.473      1.858      0.064      -0.050       1.809
occup[T.ndurman]      0.5158      0.808      0.639      0.523      -1.071       2.103
occup[T.other]        1.2398      0.828      1.497      0.135      -0.388       2.867
occup[T.profserv]    -0.5022      0.746     -0.673      0.501      -1.968       0.963
occup[T.services]    -1.8033      0.777     -2.321      0.021      -3.330      -0.277
occup[T.trade]       -1.4979      0.729     -2.054      0.040      -2.931      -0.065
occup[T.trcommpu]    -0.3448      0.913     -0.378      0.706      -2.138       1.449
educ                  0.5313      0.070      7.593      0.000       0.394       0.669
smsa                  0.7621      0.291      2.623      0.009       0.191       1.333
==============================================================================
Omnibus:                      214.125   Durbin-Watson:                   1.864
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.353
Skew:                           1.851   Prob(JB):                    1.11e-186
Kurtosis:                       8.036   Cond. No.                         216.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""